In [111]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
hltb = requests.get('https://howlongtobeat.com/game.php?id=4250').content
hltb_soup = BeautifulSoup(hltb, "lxml")
capture_title = hltb_soup.find('div', attrs = {'class': 'profile_header shadow_text'})
game_title = capture_title.get_text().strip('\n').strip(' $')
game_title

'Half-Life 2: Episode One'

In [3]:
## Need to generate numbers to find all titles, including 4 and 5 digits

In [57]:
def text_from_tag(tag):
    return tag.get_text().strip('\n').strip(' $')

In [123]:
def get_average_time(times):
    return text_from_tag(times.findAll('td')[2])

def get_median_time(times):
    return text_from_tag(times.findAll('td')[3])

def get_rushed_time(times):
    return text_from_tag(times.findAll('td')[4])

def get_leisure_time(times):
    return text_from_tag(times.findAll('td')[5])

def get_poll_count(times):
    return text_from_tag(times.findAll('td')[1])

def get_play_title(times):
    return text_from_tag(times.findAll('td')[0])


print(get_average_time(main_extra_times))
print(get_median_time(main_extra_times))
print(get_rushed_time(main_extra_times))
print(get_leisure_time(main_extra_times))
print(get_poll_count(main_extra_times))
print(get_play_title(main_extra_times))

4h 30m
4h
3h 07m
7h 45m
218
Main + Extras


In [120]:
def get_main_story_times(soup):
    return soup.findAll('tr')[1]

def get_main_story_extras_times(soup):
    return soup.findAll('tr')[2]

def get_completionist_times(soup):
    return soup.findAll('tr')[3]

def get_all_playstyles_times(soup):
    return soup.findAll('tr')[4]

get_main_story_times(hltb_soup)

<tr>
<td>Main Story</td>
<td class="center time_100_text">787 </td> <td>3h 55m </td>
<td>4h </td>
<td>2h 40m </td>
<td>6h 58m </td>
</tr>

In [150]:
def generate_row(tag):
    return pd.DataFrame(data = [[get_play_title(tag), get_poll_count(tag), get_average_time(tag),
                                 get_median_time(tag), get_rushed_time(tag), get_leisure_time(tag)]],
                       columns = ['Playstyle', 'Polled', 'Average', 'Median', 'Rushed', 'Leisure'])

def cook_soup(id_no):
    hltb = requests.get(str('https://howlongtobeat.com/game.php?id=' + id_no)).content
    hltb_soup = BeautifulSoup(hltb, "lxml")
    return hltb_soup

def cook_soup(id_no):
    return BeautifulSoup(requests.get(str('https://howlongtobeat.com/game.php?id=' + id_no)).content, "lxml")

def create_game_entry(id_no):
    soup = cook_soup(id_no)
    main = generate_row(get_main_story_times(soup))
    main_extra = generate_row(get_main_story_extras_times(soup))
    completionist = generate_row(get_completionist_times(soup))
    all_playstyles = generate_row(get_all_playstyles_times(soup))
    return main.append(main_extra).append(completionist).append(all_playstyles)


half_life_episode_two = create_game_entry("4250")
half_life_episode_two

,Playstyle,Polled,Average,Median,Rushed,Leisure
0,Main Story,787,3h 55m,4h,2h 40m,6h 58m
0,Main + Extras,218,4h 30m,4h,3h 07m,7h 45m
0,Completionists,184,5h 33m,5h,3h 23m,18h 38m
0,All PlayStyles,1.2K,4h 16m,4h,2h 46m,18h 04m


In [101]:
# add game name as a further col? Then could be a single dataframe

In [151]:
rdr2 = create_game_entry('27100')
rdr2

,Playstyle,Polled,Average,Median,Rushed,Leisure
0,Main Story,146,43h 12m,43h 45m,30h 57m,52h 14m
0,Main + Extras,289,72h 56m,70h,50h 43m,110h 44m
0,Completionists,34,137h 51m,135h 21m,103h 53m,168h 18m
0,All PlayStyles,469,68h 23m,62h,41h 49m,132h 35m
